<a href="https://colab.research.google.com/github/crispu93/PML2021/blob/main/Evaluation_metrics_and_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluation metrics and scoring
Es importante elegir la métrica correcta cuando se elige entre varios modelos y se ajustan los parámetros del modelo.

- **Ejemplo 1.**
Se tiene una aplicación para la detección de cáncer usando un test automatizado.
Si el test es negativo, el paciente se supone como sano. Si el test es positivo, el paciente es sometido a más estudios.
Dos tipos de errores: 
1. El paciente está sano y el test sale positivo
2. El paciente está enfermo y el test sale negativo

A estos errores se les conoce como falso positivo y falso negativo respectivamente.
En el ejemplo del cáncer queremos evitar tantos falsos negativos como sea posible.

- **Ejemplo 2.**
La tarea es predecir cuando un usuario de internet dará click a un item publicitario, dará click (mostrando interés) o no. La mayoría de los anuncios de internet no resultan en un click. Podría pasar que para que un usuario de click a un anuncio, a éste se le sean mostrados 100 anuncios. En otras palabras el dataset consiste de 1% (click) y 99% (no-click).

Podemos crear un clasificador con 99% de accuracy, pero eso no toma en cuenta el desbalance en las clases.

## Ejemplo. Clasificador del dígito 9

### Creación del dataset

In [1]:
from sklearn.datasets import load_digits 
from sklearn.model_selection import train_test_split

digits = load_digits()
## 1 a todos los que son 9 y 0 a los que no
y = (digits.target == 9)
X_train, X_test, y_train, y_test = train_test_split(
digits.data, y, random_state=0)

print(X_train.shape)
print(X_test.shape)

(1347, 64)
(450, 64)


### Clasificador que siempre predice la clase mayoritaria

In [2]:
import numpy as np
from sklearn.dummy import DummyClassifier

dummy_majority = DummyClassifier(strategy='most_frequent').fit(X_train, y_train) 
pred_most_frequent = dummy_majority.predict(X_test)
print("Unique predicted labels: {}".format(np.unique(pred_most_frequent)))
print("Test score: {:.2f}".format(dummy_majority.score(X_test, y_test)))

Unique predicted labels: [False]
Test score: 0.90


Se obtuvo un accuracy de 90% sin haber aprendido nada

### Clasificador con árboles de decisión

In [3]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=2).fit(X_train, y_train) 
pred_tree = tree.predict(X_test)
print("Test score: {:.2f}".format(tree.score(X_test, y_test)))

Test score: 0.92


### Clasificador que hace predicciones aleatorias pero produce clases con la misma proporción que el conjunto de entrenamiento

In [4]:
import warnings
warnings.filterwarnings('ignore')

dummy = DummyClassifier().fit(X_train, y_train)
pred_dummy = dummy.predict(X_test)
print("dummy score: {:.2f}".format(dummy.score(X_test, y_test)))

dummy score: 0.81


### Clasificador con regresión logística

In [5]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(C=0.1).fit(X_train, y_train) 
pred_logreg = logreg.predict(X_test)
print("logreg score: {:.2f}".format(logreg.score(X_test, y_test)))

logreg score: 0.98


Un clasificador aleatorio obtiene más de 80% de accuracy. Se vuelve dificil de juzgar cual de todos los resultados es realmente útil.

Acurracy puede llegar a no ser la mejor medida de desempeño en dos situaciones particulares: Cuando se tienen variables con objetivo multi clase o cuando se tienen clases desbalanceadas.

Por ejemplo, cuando se tienen 3 o más clases y un clasificador con 80% de accuracy, no podemos saber si es porque todas las clases se predicen igual de bien o hay una o dos clases que están siendo negadas por el modelo.

## Métricas basadas en la matriz de confusión

### Matriz de confusión
Provee una de las mejores formas de representar las evaluaciones de un clasificador binario

Insert image

In [6]:
from sklearn.metrics import confusion_matrix

print("Most frequent class:") 
print(confusion_matrix(y_test, pred_most_frequent)) 
print("\nDummy model:") 
print(confusion_matrix(y_test, pred_dummy)) 
print("\nDummy model:") 
print(confusion_matrix(y_test, pred_tree)) 
print("\nLogistic Regression") 
print(confusion_matrix(y_test, pred_logreg))


Most frequent class:
[[403   0]
 [ 47   0]]

Dummy model:
[[364  39]
 [ 44   3]]

Dummy model:
[[390  13]
 [ 24  23]]

Logistic Regression
[[402   1]
 [  6  41]]


1. En el primer caso solamente se predice una clase
2. En el *Dummy model* tiene un número pequeño de *true positives* y hay más *false positives* que *true positives* (revisar la última columna)
3. Las predicciones hechas con un árbol de decisión parecen ser mejores que las anteriores, pero el accuracy es parecido
4. Tiene menos falsos positivos y falsos negativos

### Relación de la matriz de confusión con accuracy

accuracy es el número de predicciones correctas dividido entre el número de muestras

$${accuracy} = \frac{TP + TN}{TP + TN + FP + FN}$$

Para resumir la información de la matriz de confusión existen otras métricas, siendo las más comunes $precision$ y $recall$




### Precision and recall

$precision$ mide cuantas de las muestras que se predijeron como positivas, son realmente positivas.

$$precision = \frac{TP}{TP + FP}$$


Se usa como medida de desempeño cuando el objetivo es limitar el número de falsos positivos. También es conocida como *positive predictive value (PPV)*

$recall$, por otro lado, mide cuantas de las muestras positivas son capturadas por prediciones positivas

$$recall = \frac{TP}{TP+FN}$$

Se usa como medida de desempeño cuando el objetivo es identificar todas las muestras positivas, es decir, evitar los falsos negativos. También se le conoce como *sensitivity* o *True Positive Rate (TPR)*

### $F_1$-score
Una forma de resumir dos medidas tan importantes como $precision$ y $recall$ es usar la media armónica, que penaliza los valores extremos

$$F_1 = \frac{2}{\frac{1}{precision}+\frac{1}{recall}}$$

Una desventaja del $F_1$-score es que es más difícil de interpretar y explicar que la $accuracy$

### Ejemplo. Clasificador del dígito 9 (continuación)

In [7]:
from sklearn.metrics import f1_score 

print("f1 score most frequent: {:.2f}".format( f1_score(y_test, pred_most_frequent)))
print("f1 score dummy: {:.2f}".format(f1_score(y_test, pred_dummy))) 
print("f1 score tree: {:.2f}".format(f1_score(y_test, pred_tree))) 
print("f1 score logistic regression: {:.2f}".format(f1_score(y_test, pred_logreg)))

f1 score most frequent: 0.00
f1 score dummy: 0.07
f1 score tree: 0.55
f1 score logistic regression: 0.92


In [8]:
from sklearn.metrics import classification_report 

print(classification_report(y_test, pred_tree, 
                            target_names=["not nine", "nine"]))

              precision    recall  f1-score   support

    not nine       0.94      0.97      0.95       403
        nine       0.64      0.49      0.55        47

    accuracy                           0.92       450
   macro avg       0.79      0.73      0.75       450
weighted avg       0.91      0.92      0.91       450



In [9]:

print(classification_report(y_test, pred_dummy, 
                            target_names=["not nine", "nine"]))

              precision    recall  f1-score   support

    not nine       0.89      0.90      0.90       403
        nine       0.07      0.06      0.07        47

    accuracy                           0.82       450
   macro avg       0.48      0.48      0.48       450
weighted avg       0.81      0.82      0.81       450



In [10]:
print(classification_report(y_test, pred_logreg, 
                            target_names=["not nine", "nine"]))

              precision    recall  f1-score   support

    not nine       0.99      1.00      0.99       403
        nine       0.98      0.87      0.92        47

    accuracy                           0.98       450
   macro avg       0.98      0.93      0.96       450
weighted avg       0.98      0.98      0.98       450

